In [144]:
from api_key import KEY
from urllib.parse import urlencode
import requests
import json

In [145]:
# KEY

In [146]:
import yapi

# Identificators

In [147]:
PLAYLIST_ID = 'PLQPfsdgAXeMlB0vNrePI1qfhjSt91yShn'

# yAPI instance

In [6]:
api = yapi.YoutubeAPI(KEY)

# Fetching playlist items

In [78]:
playlistItems = api.get_playlist_items_by_playlist_id(PLAYLIST_ID, max_results=50)

In [79]:
videos = vars(playlistItems)

In [83]:
len(videos)

5

In [84]:
videos.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'pageInfo', 'items'])

In [87]:
len(videos["items"])

50

In [88]:
videos['nextPageToken']

'CDIQAA'

In [91]:
playlistItems = api.get_playlist_items_by_playlist_id(PLAYLIST_ID, max_results=20, pageToken='CDIQAA')

TypeError: get_playlist_items_by_playlist_id() got an unexpected keyword argument 'pageToken'

In [86]:
vars(videos["items"][0]).keys()

dict_keys(['kind', 'etag', 'id', 'snippet', 'contentDetails', 'status'])

In [31]:
vars(videos["items"][0])["contentDetails"]

Namespace(videoId='xsSz-l8G-0Q', videoPublishedAt='2018-03-02T08:00:09.000Z')

# Fetching video meta data

In [32]:
video = api.get_video_info('xsSz-l8G-0Q')

In [35]:
vars(video).keys()

dict_keys(['kind', 'etag', 'pageInfo', 'items'])

In [45]:
vars(vars(video)["items"][0]).keys()

dict_keys(['kind', 'etag', 'id', 'snippet', 'contentDetails', 'status', 'statistics', 'player'])

In [62]:
vars(vars(video)["items"][0])

{'contentDetails': Namespace(caption='false', definition='hd', dimension='2d', duration='PT1H5M18S', licensedContent=True, projection='rectangular'),
 'etag': '"_gJQceDMxJ8gP-8T2HLXUoURK8c/YCTgOGNmL9b43Hs6L8Ls_TEcjcI"',
 'id': 'xsSz-l8G-0Q',
 'kind': 'youtube#video',
 'player': Namespace(embedHtml='<iframe width="480" height="270" src="//www.youtube.com/embed/xsSz-l8G-0Q" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>'),
 'snippet': Namespace(categoryId='22', channelId='UCLG5Qu54Q7gywaCeDs5etuQ', channelTitle='BALKAN INFO - Zvanični kanal', defaultAudioLanguage='sr', description='Gost emisije "Intervju" bio je gospodin Miroslav Lazanski, novinar i vojni analitičar. Diplomirao je na Pravnom fakultetu u Zagrebu, gde je i započeo novinarsku karijeru. Od 1991. godine prešao je u „Politiku“ gde i danas radi. Izveštavao je iz Sirije, Somalije, sa Krima, takođe izveštavao o ratovima u Avganistanu, Čečeniji, Kongu, Iraku, Iranu, Libanu, Jemenu i Libiji... Intervjui

In [60]:
vars(vars(video)["items"][0])["statistics"]

Namespace(commentCount='231', dislikeCount='290', favoriteCount='0', likeCount='547', viewCount='18526')

In [53]:
vars(vars(vars(video)["items"][0])["snippet"]).keys()

dict_keys(['publishedAt', 'channelId', 'title', 'description', 'thumbnails', 'channelTitle', 'tags', 'categoryId', 'liveBroadcastContent', 'localized', 'defaultAudioLanguage'])

In [54]:
vars(vars(vars(video)["items"][0])["snippet"])["title"]

'INTERVJU: Miroslav Lazanski - Srbija mora da sačuva vojnu neutralnost i vrati vojni rok! (2.3.2018)'

In [58]:
# vars(vars(vars(video)["items"][0])["snippet"])["description"]

In [63]:
vars(vars(vars(video)["items"][0])["snippet"])["publishedAt"]

'2018-03-02T08:00:09.000Z'

In [64]:
type(vars(vars(vars(video)["items"][0])["snippet"])["publishedAt"])

str

In [68]:
from datetime import datetime

datetime_object = datetime.strptime(vars(vars(vars(video)["items"][0])["snippet"])["publishedAt"].split("T")[0], '%Y-%m-%d')

In [69]:
datetime_object

datetime.datetime(2018, 3, 2, 0, 0)

In [92]:
import requests 

In [105]:
url = "https://www.googleapis.com/youtube/v3/playlistItems"
from urllib.parse import urlencode
# params["key"] = KEY
params = {
            'playlistId': PLAYLIST_ID,
            'part': 'id, snippet, contentDetails, status',
            'maxResults': 50,
            'key': KEY
        }
# urllib.urllib.urlencode

In [106]:
url = url+"?"+urlencode(params)

In [108]:
r = requests.get(url)

In [117]:
import json
json.loads(r.text).keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'pageInfo', 'items'])

In [118]:
json.loads(r.text)["nextPageToken"]

'CDIQAA'

In [119]:
json.loads(r.text)["items"][0].keys()

dict_keys(['kind', 'etag', 'id', 'snippet', 'contentDetails', 'status'])

In [124]:
json.loads(r.text)["items"][0]["id"]

'UExRUGZzZGdBWGVNbEIwdk5yZVBJMXFmaGpTdDkxeVNobi5DN0Y5ODNGRkJDMEIwNDZB'

In [125]:
url = "https://www.googleapis.com/youtube/v3/playlistItems"
from urllib.parse import urlencode
# params["key"] = KEY
params = {
            'playlistId': PLAYLIST_ID,
            'part': 'id, snippet, contentDetails, status',
            'maxResults': 50,
            'key': KEY,
        }

In [126]:
url = url+"?"+urlencode(params)

In [127]:
r = requests.get(url)

In [130]:
json.loads(r.text)["items"][0]["id"]

'UExRUGZzZGdBWGVNbEIwdk5yZVBJMXFmaGpTdDkxeVNobi4wRUE3MkE2QzM5M0M3RUMx'

In [181]:
def handle_response(url, params_dict):
    """ Sends request with specified params, returns list of items and nextPageToken
    """
    url = url+"?"+urlencode(params_dict)
    resp = requests.get(url)
#     print(resp)
    response_dict = json.loads(resp.text)
    if "nextPageToken" in response_dict:
        return_dict = {
            "items": response_dict["items"],
            "nextPageToken": response_dict["nextPageToken"]
        }
    else: 
        return_dict = {
            "items": response_dict["items"],
            "nextPageToken": None
        }
    return return_dict

In [184]:
def get_all_playlist_items(params_dict):
    """ Returns all video ID's from specified Play List
    """
    item_ids = []
    stop = False
    while stop == False:
        url = "https://www.googleapis.com/youtube/v3/playlistItems"
        if item_ids == []:
            response = handle_response(url, params_dict)
            for item in response["items"]:
                item_ids.append(item["id"])
            npt = response["nextPageToken"]
            print("FIRST NPT: {}".format(npt))
        else: 
            params_dict["pageToken"] = npt
            response = handle_response(url, params_dict)
            for item in response["items"]:
                item_ids.append(item["id"])
            npt = response["nextPageToken"]
            print("NPT: {}".format(npt))
            print(len(response["items"]))
            if npt == None:
                stop = True
    return item_ids

In [186]:
params = {
            'playlistId': PLAYLIST_ID,
            'part': 'id, snippet, contentDetails, status',
            'maxResults': 50,
            'key': KEY,
        }
len(get_all_playlist_items(params))

FIRST NPT: CDIQAA
NPT: CGQQAA
50
NPT: CJYBEAA
50
NPT: CMgBEAA
50
NPT: CPoBEAA
50
NPT: CKwCEAA
50
NPT: CN4CEAA
50
NPT: CJADEAA
50
NPT: CMIDEAA
50
NPT: None
22


472